In [1]:
from shapely.geometry import MultiPoint, LineString, MultiLineString, Point
from shapely.ops import nearest_points, linemerge
import numpy as np
# from libpysal.cg.voronoi import voronoi, voronoi_frames
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.ops import unary_union, polygonize, cascaded_union, split, polygonize_full
from shapely import affinity
from datetime import datetime
from os.path import exists
import pygeos as pg
import os


/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
df_qv = gpd.read_file('gis/SIRGAS_GPKG_quadraviariaed_2017.gpkg')
df_l = gpd.read_file(f'zip://gis/SIRGAS_SHP_logradouronbl.zip!SIRGAS_SHP_logradouronbl/SIRGAS_SHP_logradouronbl.shp')

In [3]:
df_dist = gpd.read_file(f'zip://gis/SIRGAS_SHP_distrito.zip!SIRGAS_SHP_distrito/SIRGAS_SHP_distrito_polygon.shp')
df_represas = gpd.read_file(f'zip://gis/SIRGAS_SHP_represanivelmax.zip!SIRGAS_SHP_represanivelmax').set_crs(epsg=31983)
df_massa_dagua = gpd.read_file(f'zip://gis/SIRGAS_SHP_hidrogramassadagua.zip!SIRGAS_SHP_hidromassadagua/SIRGAS_SHP_hidromassadagua_polygon.shp').set_crs(epsg=31983)

In [4]:
df_municipio = gpd.read_file('gis/limite_pmsp.gpkg')

In [5]:
df_municipio = df_municipio.set_crs(epsg=31983)

In [6]:
df_dist = df_dist.set_crs(epsg=31983)

## Poligonos de vias unidos para cada distrito

In [ ]:
for id, dist in df_dist.iterrows():
    
    file = f'resultado/pre-processamentos/{dist.ds_codigo}_poligono_de_vias_unido_{dist.ds_nome.lower().replace(" ", "_")}.gpkg'

    if exists(file):
        continue

    print(dist.ds_nome)

    qv = df_qv.clip(dist.geometry)      
#     l = df_l.clip(dist.geometry)
    represas = df_represas.clip(dist.geometry)
    represas["geometry"] = represas["geometry"].apply(lambda x: shapely.wkt.loads(shapely.wkt.dumps(x, rounding_precision=4)))
    
    massa_dagua = df_massa_dagua.clip(dist.geometry)
    massa_dagua["geometry"] = massa_dagua["geometry"].apply(lambda x: shapely.wkt.loads(shapely.wkt.dumps(x, rounding_precision=4)))

    gdf_dist = gpd.GeoDataFrame(geometry=[dist.geometry]).set_crs(epsg=31983)

    qv["geometry"] = qv["geometry"].apply(lambda x: shapely.wkt.loads(shapely.wkt.dumps(x, rounding_precision=4)))

    pvias = gpd.overlay(gdf_dist, 
            qv.explode(index_parts=True)[qv.explode(index_parts=True).geometry.geom_type == 'Polygon'],
            how='difference')

    pvias["geometry"] = pvias["geometry"].apply(lambda x: shapely.wkt.loads(shapely.wkt.dumps(x, rounding_precision=4)))

    pvias = gpd.overlay(pvias, represas, how='difference')
    pvias = gpd.overlay(pvias, massa_dagua, how='difference')
   
    pvias.to_file(file, driver="GPKG")
    
    # break

## Identificação dos pontos de cruzamento e ligação

In [7]:
tolerancia_do_cruzamento = 1.5 # era 0.6

In [8]:
df_l = df_l[~df_l.lg_tipo.isin(['PQ', 'PC', 'CV'])] 

In [10]:
extremidades = df_l.apply(lambda row: 
                    [row.geometry.interpolate(0, normalized=True),
                    row.geometry.interpolate(1, normalized=True)], axis=1)
extremidades = [item for sublist in extremidades for item in sublist]
df_extremidades = gpd.GeoDataFrame(geometry=extremidades)

In [11]:
df_extremidades

,geometry
0,POINT (324031.269 7397962.661)
1,POINT (324228.312 7397931.155)
2,POINT (340737.769 7390143.226)
3,POINT (340816.447 7390159.089)
4,POINT (325401.339 7404855.962)
...,...
431651,POINT (332598.319 7351962.263)
431652,POINT (332569.727 7385565.879)
431653,POINT (332712.266 7385628.652)
431654,POINT (337244.399 7395632.481)


In [12]:
cruzamento_centroide = [p.centroid for p in df_extremidades.buffer(tolerancia_do_cruzamento).unary_union]
df_cruzamentos = gpd.GeoDataFrame(geometry=cruzamento_centroide)

In [13]:
df_cruzamentos.set_crs(epsg=31983).to_file(f'resultado/pre-processamentos/00_cruzamentos_cidade_sao_paulo.gpkg', driver="GPKG")

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [12]:
for id, dist in df_dist.iterrows():
    
    if exists(f'resultado/pre-processamentos/{dist.ds_codigo}_pre_poligono_de_vias_{dist.ds_nome.lower().replace(" ", "_")}.gpkg'):
        continue
    
    qv = df_qv.clip(dist.geometry)
    qv = gpd.GeoDataFrame(geometry=qv.buffer(0.001))
    l = df_l.clip(dist.geometry)
    l = gpd.GeoDataFrame(geometry=l.buffer(0.001))
    
    pvias = gpd.read_file(f'resultado/pre-processamentos/{dist.ds_codigo}_poligono_de_vias_unido_{dist.ds_nome.lower().replace(" ", "_")}.gpkg')

    cruzamentos = df_cruzamentos.clip(dist.geometry)
    cruzamentos_buffer = gpd.GeoDataFrame(geometry=cruzamentos.buffer(35)).set_crs(epsg=31983)
    
    esquinas = qv.overlay(cruzamentos_buffer, how='intersection').unary_union
    esquinas = gpd.GeoDataFrame(geometry=[esquinas]).explode(index_parts=True).reset_index()

    cruzamento_esquina = gpd.sjoin(cruzamentos_buffer, esquinas, how='left', predicate='intersects')
    cruzamento_esquina = cruzamento_esquina[~cruzamento_esquina.index_right.isna()]

    cruzamento_esquina.geometry = cruzamento_esquina.centroid

    linhas = [LineString(nearest_points(l.geometry, esquinas.iloc[int(l.index_right)].geometry)) for i, l in cruzamento_esquina.iterrows()]

    gdf_linhas = gpd.GeoDataFrame(geometry=linhas).set_crs(epsg=31983)

    # Relacionando espacialmente linhas ao ponto de cruzamento
    cruzamentos_linhas = gpd.sjoin(cruzamentos, gdf_linhas, how='left' )
    # [cruzamentos_linhas]

    # Excluir as linhas que cruzam logradouros
    l_sem_cruzamento = l.overlay(gpd.GeoDataFrame(geometry=cruzamentos.buffer(0.05)), how='difference')
    gdf_linhas = gdf_linhas.drop(gpd.sjoin(gdf_linhas, l_sem_cruzamento, how='inner', predicate='crosses').index)
    
    # Excluir as linhas que cruzam quadras
    gdf_linhas = gdf_linhas.loc[gpd.sjoin(gdf_linhas, gpd.GeoDataFrame(geometry=pvias.buffer(0.10)), how='inner', predicate='within').index]

    gdf_linhas = gpd.GeoDataFrame(geometry=[g for g in linemerge(gdf_linhas.geometry.to_list())])

    gdf_linhas.to_file(f'resultado/pre-processamentos/{dist.ds_codigo}_linhas_de_separacao_{dist.ds_nome.lower().replace(" ", "_")}.gpkg', driver="GPKG")

    # Separando as vias
    cruzamentos_linhas = gpd.sjoin(gpd.GeoDataFrame(geometry=cruzamentos.buffer(10)), gdf_linhas, how='right' )
    cruzamentos_linhas = gpd.GeoDataFrame(geometry=cruzamentos_linhas.dissolve('index_left').geometry.apply(lambda x: unary_union(x.buffer(0.001))))

    pvias_recortado = pvias.overlay(cruzamentos_linhas, how='difference')
    pvias_recortado = gpd.GeoDataFrame(geometry=pvias_recortado.explode().geometry).reset_index()
    pvias_recortado.set_crs(epsg=31983)

    # TODO
    # Adicionar CODLOG e SEG de LOGRADOURO
    # Pensar na interseccao das linhas de separacao
    # Pensar na limpeza de "sujeira" nas bordas dos distritos

    pvias_recortado.to_file(f'resultado/pre-processamentos/{dist.ds_codigo}_pre_poligono_de_vias_{dist.ds_nome.lower().replace(" ", "_")}.gpkg', driver="GPKG")

    print(dist.ds_nome)

    # break

/tmp/ipykernel_172389/2898379537.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31983
Right CRS: None

  cruzamento_esquina = gpd.sjoin(cruzamentos_buffer, esquinas, how='left', predicate='intersects')
/tmp/ipykernel_172389/2898379537.py:29: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:31983

  cruzamentos_linhas = gpd.sjoin(cruzamentos, gdf_linhas, how='left' )
/tmp/ipykernel_172389/2898379537.py:34: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31983
Right CRS: None

  gdf_linhas = gdf_linhas.drop(gpd.sjoin(gdf_linha

ITAQUERA


/tmp/ipykernel_172389/2898379537.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31983
Right CRS: None

  cruzamento_esquina = gpd.sjoin(cruzamentos_buffer, esquinas, how='left', predicate='intersects')
/tmp/ipykernel_172389/2898379537.py:29: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:31983

  cruzamentos_linhas = gpd.sjoin(cruzamentos, gdf_linhas, how='left' )
/tmp/ipykernel_172389/2898379537.py:34: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31983
Right CRS: None

  gdf_linhas = gdf_linhas.drop(gpd.sjoin(gdf_linha

MANDAQUI


/tmp/ipykernel_172389/2898379537.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31983
Right CRS: None

  cruzamento_esquina = gpd.sjoin(cruzamentos_buffer, esquinas, how='left', predicate='intersects')
/tmp/ipykernel_172389/2898379537.py:29: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:31983

  cruzamentos_linhas = gpd.sjoin(cruzamentos, gdf_linhas, how='left' )
/tmp/ipykernel_172389/2898379537.py:34: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31983
Right CRS: None

  gdf_linhas = gdf_linhas.drop(gpd.sjoin(gdf_linha

...

JARAGUA


/tmp/ipykernel_172389/2898379537.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31983
Right CRS: None

  cruzamento_esquina = gpd.sjoin(cruzamentos_buffer, esquinas, how='left', predicate='intersects')
/tmp/ipykernel_172389/2898379537.py:29: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:31983

  cruzamentos_linhas = gpd.sjoin(cruzamentos, gdf_linhas, how='left' )
/tmp/ipykernel_172389/2898379537.py:34: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31983
Right CRS: None

  gdf_linhas = gdf_linhas.drop(gpd.sjoin(gdf_linha

JARDIM HELENA


/tmp/ipykernel_172389/2898379537.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31983
Right CRS: None

  cruzamento_esquina = gpd.sjoin(cruzamentos_buffer, esquinas, how='left', predicate='intersects')
/tmp/ipykernel_172389/2898379537.py:29: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:31983

  cruzamentos_linhas = gpd.sjoin(cruzamentos, gdf_linhas, how='left' )
/tmp/ipykernel_172389/2898379537.py:34: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31983
Right CRS: None

  gdf_linhas = gdf_linhas.drop(gpd.sjoin(gdf_linha

JARDIM PAULISTA
